# import statements

In [229]:
import pandas as pd
import numpy as np

# User Defined Parameters

In [230]:
risk_adv = 3
risk_free = 0.045
target_return = 0.07

# create dataframe & clean data

In [231]:
# df = pd.read_csv('./example_data.csv', header=0)
# df.index = ["Return", "Risk"]

df = pd.read_csv('./PythonCW.csv', header=0)
df = df.dropna(axis=1)
df = df.drop(columns=["Date"])
df.head()


,Nvidia,AAPL,BAESY,CJJD,AZN,SAS,TSLA,Saab AB,PFE,GME,HSBC,LVMH
0,0.0281,0.0218,0.0478,0.0909,0.0321,-0.0232,0.0817,0.0439,0.0300,0.0042,0.0231,0.0656
1,-0.0273,-0.0037,0.0151,0.2833,0.0085,0.0040,-0.0787,-0.0957,0.0097,0.0067,0.0273,-0.0023
2,-0.0028,0.0377,0.0339,-0.2727,0.0110,0.0039,0.0811,-0.0256,0.0096,0.0538,0.0048,0.0158
3,0.0394,0.0489,0.0247,0.0000,0.0127,0.0078,0.0665,-0.0189,0.0000,0.1002,0.0235,0.0415
4,-0.0183,-0.0175,-0.0115,-0.0179,-0.0086,0.1089,0.1087,-0.0017,0.0027,-0.0282,-0.0469,-0.0114


# Specify tickers

In [232]:

tickers = {
    # 1: "Artemis Corporate Bond",
    # 2: "Royal London Corporate Bond",
    # 3: "Legal & General All Stocks Gilt Index Trust",
    # 4: "iShares Corporate Bond Index",
    # 5: "JPM Emerging Markets",
    # 6: "CT European Select",
    # 7: "Barings Europe Select",
    # 8: "Polar Capital European ex-UK Income",
    # 9: "BlackRock Continental European Income",
    # 10: "Legal & General European Index",
    # 11: "Rathbone Global Opportunities",
    # 12: "Baillie Gifford Managed",
    1: "Nvidia",
    2: "AAPL",
    3: "BAESY",
    4: "CJJD",
    5: "AZN",
    6: "SAS",
    7: "TSLA",
    8: "Saab AB",
    9: "PFE",
    10: "GME",
    11: "HSBC",
    12: "LVMH"
    }

# Ask user what columns they want

desired_columns = [1, 2, 3]
desired_tickers = [ tickers[i] for i in desired_columns ]
df = df[desired_tickers]

df.head()

# df.selected

,Nvidia,AAPL,BAESY
0,0.0281,0.0218,0.0478
1,-0.0273,-0.0037,0.0151
2,-0.0028,0.0377,0.0339
3,0.0394,0.0489,0.0247
4,-0.0183,-0.0175,-0.0115


# Calculate expected returns (mean 👎)

In [233]:
ex_returns = df.mean()

ex_returns.head()

Nvidia    0.012052
AAPL      0.005649
BAESY     0.003030
dtype: float64

# Calculate Standard Deviation

In [234]:
vol = df.std()
vol.head()

Nvidia    0.059409
AAPL      0.037194
BAESY     0.035482
dtype: float64

# Calculate Correlation

In [235]:
corr = df.corr()
corr.head()

,Nvidia,AAPL,BAESY
Nvidia,1.000000,0.488853,0.267376
AAPL,0.488853,1.000000,0.260428
BAESY,0.267376,0.260428,1.000000


# Calculate Covariance

In [236]:
cov = corr.mul(vol).transpose().mul(vol).transpose()
cov.head()

,Nvidia,AAPL,BAESY
Nvidia,0.003529,0.001080,0.000564
AAPL,0.001080,0.001383,0.000344
BAESY,0.000564,0.000344,0.001259


# Calculate inv covariance

In [237]:
inv_cov = np.linalg.inv(cov)
inv_cov

array([[ 382.88720335, -275.04104375,  -96.32709326],
       [-275.04104375,  973.0234089 , -142.49906576],
       [ -96.32709326, -142.49906576,  876.32616144]])

# Calculate L
#### For each asset expected return x cov^-1

e.g nvida & apple
mean(nvid) x cov^-1(appl&nvid) + mean(appl) x cov^-1(appl&nvid)

e.g. nvida & apple & baesy
mean(nvid) x cov^-1(appl&nvid) + mean(appl) x cov^-1(appl&nvid)
mean(bae) x cov^-1(appl&bae) + mean(appl) x cov^-1(appl&bae)
mean(nvid) x cov^-1(appl&nvid) + mean(appl) x cov^-1(appl&nvid)

In [238]:
l_matrix = ex_returns @ inv_cov
print(l_matrix)


[2.76882511 1.75016799 0.68946043]


# Calculate M

In [239]:
user_vector = 1
vector = pd.Series([user_vector]*len(desired_columns))

m_matrix = vector @ inv_cov

m_matrix



array([ 11.51906634, 555.48329939, 637.50000241])

# Calculate A

In [240]:
a = vector @ l_matrix
a

5.208453532660908

# Calculate B

In [241]:
b = ex_returns @ l_matrix
b

0.04534481731712773

# Calculate C

In [242]:
c = vector @ m_matrix
c

1204.502368140366

# Calculate D

In [243]:
d = b * c - (a**2)
d

27.489951639484733

# Calculate G

In [244]:
g = 1/(d) * ( (m_matrix*b) - (l_matrix*a) )
g

array([-0.50560173,  0.58467255,  0.92092918])

# Calculate H

In [245]:
h = 1/(d) * ( (l_matrix*c) - (m_matrix*a) )
h

array([119.13661861, -28.56052553, -90.57609308])

# Calculate G+H

In [246]:
gh = g + h
gh

array([118.63101689, -27.97585298, -89.6551639 ])

# Calculate Optimum Weights

In [247]:
optimum_weights = g +( target_return * h )
optimum_weights

array([ 7.83396158, -1.41456424, -5.41939734])

# Calculate Risk/Return for Optimum Weights

In [248]:
return_optimum_point = optimum_weights @ ex_returns
risk_optimum_point = (optimum_weights.T @ cov @ optimum_weights) ** (1/2)

print(return_optimum_point)
print(risk_optimum_point)

0.07000000000000002
0.43568672380631446


# Calculate Risk/Return for G, H and G+H

In [249]:
return_g = g @ ex_returns
risk_g = (g.T @ cov @ g) ** (1/2)

return_h = h @ ex_returns
risk_h = (h.T @ cov @ h) ** (1/2)

return_gh = gh @ ex_returns
risk_gh = (gh.T @ cov @ gh) ** (1/2)

print(f"risk_g=\t\t{risk_g}\nrisk_h=\t\t{risk_h}\nrisk_g+h=\t{risk_gh}")

risk_g=		0.04061410061441259
risk_h=		6.619372802126952
risk_g+h=	6.59081259915595


# Calculate Variance Portfolio Returns

In [250]:
#compute minimum variance portfolio return
min_var = a/c
#compute optimum variance portfolio return
opt_var = min_var - (d/(c**2))/(risk_free - min_var)

print(min_var)
print(opt_var)

0.004324153833505742
0.0038583296158371974


# Some shitty for loop

In [254]:
portfolio_returns = []
portfolio_risk = []
portfolio_weights = []
portfolio_utility = []
portfolio_sharpe_ratios = []

p_returns = np.linspace(0, 1, 101)

for i in p_returns:

    p_weights=g+(i*h)
    p_return = np.sum(p_weights * ex_returns)
    portfolio_variance = np.dot(p_weights.T, np.dot(cov, p_weights))
    p_risk = (p_weights @ cov @ p_weights.T)**0.5
    excess_return = p_return - risk_free
    p_sharpe_ratios = excess_return / p_risk
    p_utility = p_return - (0.5 * risk_adv * portfolio_variance)


    portfolio_weights.append(p_weights)
    portfolio_risk.append(p_risk)
    portfolio_utility.append(p_utility)
    portfolio_sharpe_ratios.append(p_sharpe_ratios)
    portfolio_returns.append(p_return)

portfolio_returns = np.array(portfolio_returns)
portfolio_risk = np.array(portfolio_risk)
portfolio_weights = np.array(portfolio_weights)
portfolio_utility = np.array(portfolio_utility)
#portfolio_sharpe_ratios=np.array(portfolio_sharpe_ratios)

max_sharpe = max(portfolio_sharpe_ratios)

portfolio_capital_allocations = []

for i in p_returns:
    p_weights=g+(i*h)
    p_risk = (p_weights @ cov @ p_weights.T)**0.5

    p_capital_allocation = risk_free + (max_sharpe * p_risk)

    portfolio_capital_allocations.append(p_capital_allocation)

portfolio_capital_allocations = np.array(portfolio_capital_allocations)

weight_columns = [f'{desired_tickers[i]} Weight' for i in range(len(desired_columns))]
data = {
            'Return': [p_ret for p_ret in portfolio_returns],
            'Volatility': [risk for risk in portfolio_risk],
            'Sharpe Ratio': [sharpe for sharpe in portfolio_sharpe_ratios],
            'Utility': [utility for utility in portfolio_utility],
            'Capital Allocation Line Ret': [capital for capital in portfolio_capital_allocations]
        }

for i, col in enumerate(weight_columns):
    data[col] = [w[i] for w in portfolio_weights]

print(max_sharpe)


df = pd.DataFrame(data)

df["Capital Allocation Line Ret"] = risk_free + ( max_sharpe * df["Volatility"] )

excel_file = 'portfolio_analysis.xlsx'
df.to_excel(excel_file, index=False)
print(f"Data saved to {excel_file}")


# plt.figure(figsize=(10, 6))
# plt.plot(portfolio_risk, portfolio_returns, )
# plt.title('Efficient Frontier')
# plt.xlabel('Risk (Standard Deviation)')
# plt.ylabel('Return')
# plt.show()    

0.14489867305926762
Data saved to portfolio_analysis.xlsx
